# 简介
本文使用PaddleHub模型库中的`deeplabv3p_xception65_humanseg`预训练模型，对健身和电影视频里的人物进行抠图，然后融合在一个宇宙纪录片的背景中，生成gif动画，或者附上配乐或配音，实现“太空广播操”和“宇宙演讲台”的创意。

## 关于本项目
> 针对项目还存在的改进空间、更有趣的Paddlehub玩法，希望大家多交流观点、介绍经验，共同学习进步。[个人主页](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/90149)

## 演示效果
![file](https://ai-studio-static-online.cdn.bcebos.com/fa5f09d3d89c41c29f6279bcef7f52c43004363f430a4fc98727b861833ca19a)




In [1]:
import IPython
IPython.display.Video('videos/out_gym_final.mp4')

# 视频和音频的基本概念
## 视频
视频其实就是一系列的图像在时间维度上的排列，每一个视频帧就是一张图像。
[百科链接](https://baike.baidu.com/item/%E8%A7%86%E9%A2%91%E5%B8%A7%E7%8E%87)
> 每秒的帧数(fps)或者说帧率表示图形处理器处理场时每秒钟能够更新的次数。高的帧率可以得到更流畅、更逼真的动画。一般来说30fps就是可以接受的，但是将性能提升至60fps则可以明显提升交互感和逼真感，但是一般来说超过75fps一般就不容易察觉到有明显的流畅度提升了。如果帧率超过屏幕刷新率只会浪费图形处理的能力，因为监视器不能以这么快的速度更新，这样超过刷新率的帧率就浪费掉了。

下图显示了一个视频文件的基本信息：
![file](https://ai-studio-static-online.cdn.bcebos.com/851ac0abe182442790a4c40646e0bf766341cff042a740e2a0d27b9029d263e0)

## 比特率
[百科链接](https://baike.baidu.com/item/%E6%AF%94%E7%89%B9%E7%8E%87/1022775)
> 比特率是指每秒传送的比特(bit)数。单位为 bps(Bit Per Second)，比特率越高，每秒传送数据就越多，画质就越清晰。声音中的比特率是指将模拟声音信号转换成数字声音信号后，单位时间内的二进制数据量，是间接衡量音频质量的一个指标。 视频中的比特率（码率）原理与声音中的相同，都是指由模拟信号转换为数字信号后，单位时间内的二进制数据量。

## 音频采样率
[百科链接](https://baike.baidu.com/item/%E9%9F%B3%E9%A2%91%E9%87%87%E6%A0%B7%E7%8E%87/9023551)
> 音频采样率是指录音设备在一秒钟内对声音信号的采样次数，采样频率越高声音的还原就越真实越自然。在当今的主流采集卡上，采样频率一般共分为11025Hz、22050Hz、24000Hz、44100Hz、48000Hz五个等级，11025Hz能达到AM调幅广播的声音品质，而22050Hz和24000HZ能达到FM调频广播的声音品质，44100Hz则是理论上的CD音质界限，48000Hz则更加精确一些。

# 加载工具包

In [ ]:
# 下载预训练模型
!hub install deeplabv3p_xception65_humanseg

2020-06-01 21:45:02,682-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2020-06-01 21:45:03,079-INFO: generated new fontManager
[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmpwbco8dd8/deeplabv3p_xception65_humanseg
[==================================================] 100.00%


In [ ]:
import paddlehub as hub #如果首次使用需要先安装飞桨框架和工具库，具体方法可以查看百度飞桨官网，非常简单
from PIL import Image
import os
import cv2
from tqdm import tqdm

In [ ]:
# 创建存放图片的文件夹
!mkdir images
!mkdir images/gym
!mkdir images/earth
!mkdir images/The_Matrix
!mkdir images/gymseg_out
!mkdir images/gymmerge_out
!mkdir images/speechseg_out
!mkdir images/speechmerge_out

In [ ]:
# 创建存放音视频的文件夹
!mkdir videos

# 截取每一帧的视频
> 视频数据与图像数据非常类似，都是由像素点组成的数据。在视频数据在非音频部分基本上可以视为多帧（张）图像数据的拼接，即三维图像的组合。由于视频数据与图像数据的相似性，在视频领域任务中大都可以借助图像方法来完成。

从视频中抽帧有很多种方法，本文Python OpenCV库和FFmpeg命令行两种方法。

## 使用OpenCV库抽取视频帧
`a5.mp4`是一个健身视频片段，`data/data38147/earth.mp4`文件则是太空纪录片的一个小片段，下面演示如何使用Python OpenCV库进行视频抽帧。

In [ ]:
# 定义一个抽帧函数
def cut_frames(INPUT_FILE, OUTPUT_PATH, start_frame, end_frame):
    reader = cv2.VideoCapture(INPUT_FILE)
    # 视频帧的宽度
    width = int(reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    # 视频帧的高度
    height = int(reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # 确认是否读取到视频流  
    print(reader.isOpened())
    have_more_frame = True
    c = 0
    while have_more_frame:
        have_more_frame, frame = reader.read()
        c += 1
        if c>= start_frame and c<= end_frame:
            cv2.imwrite('%s%d.png'%(OUTPUT_PATH,c), frame)
        if c>end_frame:
            print('completely!')
            break
    
    reader.release()
    cv2.destroyAllWindows()

In [ ]:
# 输入文件
# INPUT_FILE = 'a5.mp4'
# 输出图片位置
# OUTPUT_PATH = 'images/gym/'
# 开始视频帧
# start_frame = 0
# 结束视频帧
# end_frame = 480
# 开始抽帧
cut_frames('a5.mp4', 'images/gym/', 0, 480)

True
completely!


In [ ]:
# 太空背景抽帧
cut_frames('data/data38147/earth.mp4', 'images/earth/', 0, 480)

True
completely!


## 使用FFmpeg抽帧
[FFmpeg](http://ffmpeg.org/documentation.html)是一套音视频多媒体处理开源框架，它提供了对音视频的采集、编码、解码、转码、音视频分离、合并、流化、过滤器等丰富的功能。目前，业界的视频播放主要有三种架构：MPC， MPlayer和VLC，占据市场90%的份额。而三个架构均使用或者融合FFmpeg的视频解码技术。FFmpeg堪称多媒体领域的神器。
在FFmpeg中，视频抽帧只需要一行命令：
```bash
ffmpeg -i 输入视频.mp4 -r 30 输出目录/%d.png
```
具体的可选参数：
- -r 指定抽取的帧率，即从视频中每秒钟抽取图片的数量。1代表每秒抽取一帧。
- -t，表示取t秒时间的帧
- -ss，表示截取帧初始时间
- -vframes，表示截取多少帧

In [ ]:
# -r 指定抽取的帧率，即从视频中每秒钟抽取图片的数量。1代表每秒抽取一帧。
# !ffmpeg -i data/data38254/hkdg.mp4 -r 30 images/The_Matrix/%d.png
# 截取前480帧
!ffmpeg -i data/data38254/hkdg.mp4 -r 30 -vframes 480 images/The_Matrix/%d.png

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

# 人像抠图

In [ ]:
import os
def humanseg(images,output_dir):  
    #装载预训练模型
    module = hub.Module(name="deeplabv3p_xception65_humanseg")
    result = module.segmentation(data={"image":images}, output_dir=output_dir)
    # print (result)
    return result


def file_list(listdir):
    '''
    读取listdir文件夹里的序号图片文件，将名字形成im_list
    '''
    im_list=[]
    files = os.listdir(listdir)
    for i in range(len(files)) :
        im_list.append('%s%d.png' %(listdir,i+1) )
    return im_list

In [ ]:
# 健身照片批量抠图
humanseg(file_list('images/gym/'),'images/gymseg_out')

In [ ]:
# 演讲照片批量抠图
humanseg(file_list('images/The_Matrix/'),'images/speechseg_out')

# 视频融合
这里参考[PaddleHub创意赛：当维密天使来到东北](https://aistudio.baidu.com/aistudio/projectdetail/446851)项目的做法，利用PIL库的alpha_composite将两组图片融合逐一融合

## 使用FFmpeg合成gif
### 去掉视频的logo
语法：`-vf delogo=x:y:w:h[:t[:show]]`
- `x:y` 离左上角的坐标
- `w:h` logo的宽和高
- `t:` 矩形边缘的厚度默认值4
- `show` 若设置为1有一个绿色的矩形，默认值0。
### 截取时间设置
- `-ss` 开始时间
- `-t` 总共截取时间
- `-r` 每秒帧数
### 缩小输出
语法： `-vf scale=w:h`

In [ ]:
# ffmpeg将图片合成为mp4，从第15秒开始截取2秒视频，去掉视频logo
!ffmpeg -ss 00:00:15 -t 00:00:02 -r 15 -i images/gymmerge_out/%d.png -vf delogo=1650:0:220:120:100:0 out_gym.mp4

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

In [ ]:
# 将mp4转换为gif，并将动图分辨率缩放到384*216
!ffmpeg -i out_gym.mp4 -vf scale=384:216 out_gym.gif

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

## PIL库的alpha_composite图片融合
- 将太空背景与健身者抠图一一融合
- 将太空背景与演讲者抠图的前300张图片一一融合（演讲者视频只有前10秒有人物，300多张后的抠图是空白。）

1. 健身者

In [ ]:
def alpha_composite(fore_image,base_image,output_dir):
    '''
    将前景图片fore_image与背景图片base_image融合，输出到文件夹output_dir
    '''
    #利用RGBA的方式读取图片
    base_image_op = Image.open(base_image).convert("RGBA")
    fore_image_op = Image.open(fore_image).convert("RGBA")
    # print (base_image_op)
    # print (fore_image_op)

    #调整照片的size，因为背景图片较小，所以我调整背景图片来适应前景图片
    base_image_op = base_image_op.resize(fore_image_op.size)  
    # base_image = base_image.rotate(90)  #如果图片角度不同，还可以旋转图片
    
    #将前景图片与背景图片融合
    image_c = Image.alpha_composite(base_image_op, fore_image_op)
    image_c.save(output_dir + fore_image.replace('images/gymseg_out/','') )

    return image_c
# 视频转化图片的文件夹
files_v = os.listdir('images/earth/') 
#人像抠图的文件夹
fore_path='images/gymseg_out/'
#背景图片
base_dir='images/earth/'
#合成图片输出的文件夹
output_dir='images/gymmerge_out/'

for i in range(len(files_v)):
    base_image='%s%d.png'%(base_dir,i+1)
    fore_image='%s%d.png'%(fore_path,i+1)
    alpha_composite(fore_image,base_image,output_dir)

2. 演讲者

In [ ]:
def alpha_composite(fore_image,base_image,output_dir):
    '''
    将前景图片fore_image与背景图片base_image融合，输出到文件夹output_dir
    '''
    #利用RGBA的方式读取图片
    base_image_op = Image.open(base_image).convert("RGBA")
    fore_image_op = Image.open(fore_image).convert("RGBA")
    # print (base_image_op)
    # print (fore_image_op)

    #调整照片的size，因为背景图片较小，所以我调整背景图片来适应前景图片
    base_image_op = base_image_op.resize(fore_image_op.size)  
    # base_image = base_image.rotate(90)  #如果图片角度不同，还可以旋转图片
    
    #将前景图片与背景图片融合
    image_c = Image.alpha_composite(base_image_op, fore_image_op)
    image_c.save(output_dir + fore_image.replace('images/speechseg_out/','') )

    return image_c
# 视频转化图片的文件夹
files_v = os.listdir('images/earth/') 
#人像抠图的文件夹
fore_path='images/speechseg_out/'
#背景图片
base_dir='images/earth/'
#合成图片输出的文件夹
output_dir='images/speechmerge_out/'

for i in range(len(files_v)):
    # 只融合前300张图片
    if i <= 300:
        base_image='%s%d.png'%(base_dir,i+1)
        fore_image='%s%d.png'%(fore_path,i+1)
        alpha_composite(fore_image,base_image,output_dir)

### 使用OpenCV库图片合成视频

### 太空健身操

In [ ]:
# 视频转化图片的文件夹
def png2video(image_dir,out_video):
    '''
    将文件夹image_dir中的每一帧图片合成视频out_video
    '''
    #fourcc代表视频流的合成格式
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #以(1280,720)的分辨率，25帧/s的速度，fourcc的视频流格式来合成视频out_video
    out = cv2.VideoWriter(out_video,fourcc, 25.0, (1280,720))
    files = os.listdir(image_dir)

    files_v = os.listdir('images/gymmerge_out/') 

    for i in range(len(files_v)): 
        image='%s%d.png' %(image_dir,i)
        
        try:
            img = cv2.imread(image)
            img = cv2.resize(img,(1280,720))
            out.write(img)#保存帧

        except:
            continue

    out.release() #关闭视频
# 使用OpenCV库进行视频融合
png2video('images/gymmerge_out/','videos/out_gym.mp4')

#### 提取广播体操音频并合成
- `9.mp3`是广播体操音频文件

In [55]:
# 提取音频
# !ffmpeg -i 9.mp3 -ss 00:01:00 -t 00:00:18 -acodec copy videos/output9.mp3

# 去掉logo
!ffmpeg -i videos/out_gym.mp4 -vf delogo=1650:0:220:120:100:0 videos/out_gym_offlogo.mp4

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

In [56]:
# 添加音频合成视频
!ffmpeg -y -i videos/out_gym_offlogo.mp4 -i videos/output9.mp3 -vcodec copy -acodec copy videos/out_gym_final.mp4

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

### 宇宙演讲者

In [ ]:
# 视频转化图片的文件夹
def png2video(image_dir,out_video):
    '''
    将文件夹image_dir中的每一帧图片合成视频out_video
    '''
    #fourcc代表视频流的合成格式
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #以(1280,720)的分辨率，25帧/s的速度，fourcc的视频流格式来合成视频out_video
    out = cv2.VideoWriter(out_video,fourcc, 25.0, (1280,720))
    files = os.listdir(image_dir)   
    # 视频转化图片的文件夹
    files_v = os.listdir('images/speechmerge_out/') 

    for i in range(len(files_v)): 
        image='%s%d.png' %(image_dir,i)
        
        try:
            img = cv2.imread(image)
            img = cv2.resize(img,(1280,720))
            out.write(img)#保存帧

        except:
            continue

    out.release() #关闭视频
# 使用OpenCV库进行视频融合
png2video('images/speechmerge_out/','videos/out_speech.mp4')

#### 合成音频

In [ ]:
# 分离原视频的音频流，注意，这里只提取12秒的音频
!ffmpeg -i data/data38254/hkdg.mp4 -ss 00:00:00 -t 00:00:12 videos/speech.mp3 

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

In [ ]:
# 添加音频合成视频
!ffmpeg -i videos/out_speech.mp4 -i videos/speech.mp3  -vcodec copy -acodec copy videos/out_speech_final.mp4 -loglevel quiet

# 静态效果演示

![file](https://ai-studio-static-online.cdn.bcebos.com/d3d8807ab70c4f4680acad6155bbf464bb58291078e3440588800b36adc159dd)


![file](https://ai-studio-static-online.cdn.bcebos.com/bdf0f5916dad4f67b490c8579b42cff499d7b3f7947e4b689ba63c714ada89c1)
